In [15]:
# youssef ahmed ibrahim 
# 223101109

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from tqdm import tqdm

import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np

In [17]:
class FaceEmotionsDataset(Dataset):
    def __init__(self, dataset_path, transform=None):
        self.dataset = ImageFolder(root=dataset_path, transform=transform)
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        image, label = self.dataset[index]
        return image, label

    @property
    def classes(self):
        return self.dataset.classes

In [18]:

path = r"A:\future\collage\sem 5\NN\assignment\1\archive\Data"

In [19]:
train_transform = transforms.Compose([
    transforms.Resize((48, 48)),
    transforms.Grayscale(1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

test_transform = transforms.Compose([
    transforms.Resize((48, 48)),
    transforms.Grayscale(1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

temp_dataset = FaceEmotionsDataset(dataset_path=path, transform=None)
total_len = len(temp_dataset)
train_len = int(0.8 * total_len)
test_len = total_len - train_len

indices = list(range(total_len))
np.random.seed(42)
np.random.shuffle(indices)
train_indices = indices[:train_len]
test_indices = indices[train_len:]

train_dataset_full = FaceEmotionsDataset(dataset_path=path, transform=train_transform)
test_dataset_full = FaceEmotionsDataset(dataset_path=path, transform=test_transform)

train_dataset = Subset(train_dataset_full, train_indices)
test_dataset = Subset(test_dataset_full, test_indices)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=0, pin_memory=True)
test_loader  = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=0, pin_memory=True)

print(f"Total samples: {total_len}")
print(f"Training samples: {train_len}")
print(f"Testing samples: {test_len}")
print(f"Number of classes: {len(temp_dataset.classes)}")
print(f"Classes: {temp_dataset.classes}")

Total samples: 59099
Training samples: 47279
Testing samples: 11820
Number of classes: 5
Classes: ['Angry', 'Fear', 'Happy', 'Sad', 'Suprise']


In [20]:
class MLPClassifier(nn.Module):
    def __init__(self, num_classes, input_size=48*48):
        super().__init__()
        
        self.model = nn.Sequential(
            nn.Flatten(),
            
            nn.Linear(input_size, 1024),
            nn.ReLU(),
            
    
            nn.Linear(1024, 512),
            nn.ReLU(),
            
            nn.Linear(512, 256),
            nn.ReLU(),
            
            nn.Linear(256, 128),
            nn.ReLU(),
            
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.model(x)

In [21]:
num_classes = len(temp_dataset.classes)
model = MLPClassifier(num_classes=num_classes, input_size=48*48)

print(model)
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

MLPClassifier(
  (model): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=2304, out_features=1024, bias=True)
    (2): ReLU()
    (3): Linear(in_features=1024, out_features=512, bias=True)
    (4): ReLU()
    (5): Linear(in_features=512, out_features=256, bias=True)
    (6): ReLU()
    (7): Linear(in_features=256, out_features=128, bias=True)
    (8): ReLU()
    (9): Linear(in_features=128, out_features=5, bias=True)
  )
)

Total parameters: 3,049,989
Trainable parameters: 3,049,989


In [22]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
print(f"Using device: {device}")

Using device: cpu


In [23]:
def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in tqdm(train_loader, desc="Training"):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    return epoch_loss, epoch_acc

In [24]:
def evaluate(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Evaluating"):
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(test_loader)
    epoch_acc = 100 * correct / total
    return epoch_loss, epoch_acc


In [25]:
num_epochs = 5
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []

best_test_acc = 0.0

print("Starting training with MLP...\n")

for epoch in range(num_epochs):
    print(f"Epoch [{epoch+1}/{num_epochs}]")
    
    # Train
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    
    # Evaluate
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)
    test_losses.append(test_loss)
    test_accuracies.append(test_acc)
    
    
    if test_acc > best_test_acc:
        best_test_acc = test_acc
    
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")
    print(f"Best Test Acc: {best_test_acc:.2f}%\n")

print("Training completed!")
print(f"Final Best Test Accuracy: {best_test_acc:.2f}%")


Starting training with MLP...

Epoch [1/5]


Evaluating: 100%|██████████| 93/93 [00:15<00:00,  5.83it/s]


Train Loss: 1.4783, Train Acc: 36.63%
Test Loss: 1.4369, Test Acc: 40.17%
Best Test Acc: 40.17%

Epoch [2/5]


Evaluating: 100%|██████████| 93/93 [00:15<00:00,  5.89it/s]


Train Loss: 1.4220, Train Acc: 40.23%
Test Loss: 1.3917, Test Acc: 41.95%
Best Test Acc: 41.95%

Epoch [3/5]


...

Train Loss: 1.3842, Train Acc: 42.39%
Test Loss: 1.3856, Test Acc: 42.04%
Best Test Acc: 42.04%

Epoch [4/5]


Evaluating: 100%|██████████| 93/93 [00:15<00:00,  5.90it/s]


Train Loss: 1.3655, Train Acc: 43.37%
Test Loss: 1.3659, Test Acc: 43.27%
Best Test Acc: 43.27%

Epoch [5/5]


Evaluating: 100%|██████████| 93/93 [00:16<00:00,  5.64it/s]

Train Loss: 1.3423, Train Acc: 44.74%
Test Loss: 1.3363, Test Acc: 44.37%
Best Test Acc: 44.37%

Training completed!
Final Best Test Accuracy: 44.37%
